Drew Lickman\
CSCI 4820-001\
Project #1\
Due: 9/9/24

# Minimum Distance Edit Algorithm

## Assignment Requirements:

### Input
---

- [words.txt](words.txt) is the input, which holds **lowercase** sets of words on each line.
    - The first word of each line is the <u>target</u>, and all the other words in the line are <u>source</u> words that will transform into the target
- Sample input files
    - [costs.csv](costs.csv) uses Levenshtein substitution costs
    - [costs2.csv](costs2.csv) uses confusion matrix substitution costs

### Processing
---

- Insertions and Deletions cost 1
- Substitution costs are read from the costs.csv files
- For each pair of source and target words, use the Minimum Edit Distance algorithm (using both cost methods)
    - Then output the backtrace of operations (K I S D)
    - Must be able to capture all possible sources for the minimum cost at each cell
    - Randomly select one of the possible cells that provide the minimum cost
    - Do NOT seed random number generator

### Output
---

- 4 lines per method (costs and costs2)
1. Source word
2. Vertical bar for each operation per character
3. Target word
4. Operation for each character and sum of edit cost 
    - k = keep
    - i = insert
    - s = substitute
    - d = delete
- 50 hyphens will separate a pair of words from the next pair

## Python Code

In [957]:
# This block of code processes words.txt and defines the targets and sources
targets = []
sources = []
pairTargSources = []

costsFileName = "costs.csv"
costs2FileName = "costs2.csv"
costMethods = [costsFileName, costs2FileName]

# Read & save the target and source words from words.txt
with open("words.txt") as wordList:
    lines = wordList.readlines()

    for line in lines:
        currentLine = line.split()      # Split the line into words
        targets.append(currentLine[0])  # Target word is first in each line
        sources.append(currentLine[1:]) # Source words are everything after the first word of the line
                                        # Also, the [1:] saves it as an array, even if it's just one source word

for row in range(len(targets)):
    pairTargSources.append([targets[row], sources[row]]) # Explicitly pair the sources to their target

print("targets:", targets)
print("sources:", sources)
print("\nTargets with matching sources: ")
for pair in pairTargSources:
    print(pair)


targets: ['duped', 'mischievous', 'execution', 'tagctatcacgaccgcggtcgatttgcccgac']
sources: [['duked'], ['mischief', 'devious'], ['intention'], ['aggctatcacctgacctccaggccgatgccc']]

Targets with matching sources: 
['duped', ['duked']]
['mischievous', ['mischief', 'devious']]
['execution', ['intention']]
['tagctatcacgaccgcggtcgatttgcccgac', ['aggctatcacctgacctccaggccgatgccc']]


In [958]:
# This block of code reads the CSV files and calculates the cost of substituting letters

alphabet = {letter: index for index, letter in enumerate("abcdefghijklmnopqrstuvwxyz")}
substitutionCost = []
substitutionCost2 = []
# Function able to work with costs.csv and costs2.csv (not working, doing manual override)
# Can improve by saving file first time it's read
def readCostFromCSV(file, substitutionTable):
    with open(file) as costList:
        costLines = costList.readlines()

        for costLine in costLines:
            currentCostLine = costLine.split(",") # Split each value by commas
            substitutionTable.append(currentCostLine) # Save substitution cost to array, indexed by 2D alphabet
        substitutionTable.pop(0) # Remove the first line of cost.csv

        # Cleanup
        for letter in substitutionTable:
            letter.pop(0) # Remove the letter from each cost array
            #letter[-1].rstrip() # Remove the newlines  #I don't think this is working

    # Accessible with alphabet dictionary
    #print(substitutionCost[alphabet["a"]][alphabet["i"]])
    return substitutionTable # Return the entire cost 2D array

costsCSV = readCostFromCSV(costsFileName, substitutionCost)
costs2CSV = readCostFromCSV(costs2FileName, substitutionCost2) #Manual, since it's only two methods
costCSVs = [costsCSV, costs2CSV]

# Returns either Levenshtein cost or Confusion Matrix cost
def getCostFromCSV(csv, letter1, letter2):
    #print("costsCSV table: ", costsCSV)
    #print("costs2CSV table: ", costs2CSV)
    #print()
    intCost = int(csv[alphabet[letter1]][alphabet[letter2]]) # Calculate the specific substitution cost
    return intCost

print("costsCSV a -> i: ", getCostFromCSV(costsCSV, "a", "i")) # "a" substitution TO "i" returns 2
print("costs2CSV a -> i: ", getCostFromCSV(costs2CSV, "a", "i")) # "a" substitution TO "i" returns 118


costsCSV a -> i:  2
costs2CSV a -> i:  118


In [959]:
# Helper function

def operation(string, file, letter1, letter2):
    match(string):
        case "keep":
            return 0
        case "insert":
            return 1
        case "delete":
            return 1
        case "substitute":
            if (file is not None) and (letter1 is not None) and (letter2 is not None):
                cost = getCostFromCSV(file, letter1, letter2)
                return cost
            return -100
        case _:
            return -200

print("Substitution: ", operation("substitute", costsCSV, "a", "i")) #debug

Substitution:  2


In [960]:
# Minimum Edit Distance Algorithm

#TestTable = []

def MED(source, target, costMethod):
    #save the source and target words for usage in backtrace
    global savedSource, savedTarget
    savedSource = source
    savedTarget = target

    sourceLen = len(source)
    targetLen = len(target)
    #print("Table dimensions: ", sourceLen, targetLen)
    DistTable = [[0 for i in range(targetLen+1)] for j in range(sourceLen+1)] # 2D array the size of source length+1 by target length+1

    # Initialize table size and number axies
    for i in range(1, sourceLen+1):
        DistTable[i][0] = DistTable[i-1][0] + operation("delete", "", "", "") #cols
    for j in range(1, targetLen+1):
        DistTable[0][j] = DistTable[0][j-1] + operation("insert", "", "", "") #rows

    # Recurrence relation:
    for row in range(1, sourceLen+1):
        for col in range(1, targetLen+1):
            # How will I know which operation is performed?
            tryDelete = DistTable[row-1][col] + operation("delete", "", "", "")#delete, #1 #delete(source[i])?
            trySubstitute = DistTable[row-1][col-1] + operation("substitute", costMethod, source[row-1], target[col-1]) #0 if same or 2 if different
            tryInsert = DistTable[row][col-1] + operation("insert", "", "", "")#insert) #1 #insert(target[j])?

            DistTable[row][col] = min(
                            tryDelete,
                            trySubstitute,
                            tryInsert)

    printing = True #debug: displays visual table
    if printing:
        # Print table
        print("      ", end="")
        # Print the target on top
        for letter in target: 
            print(letter, end="  ")
        print()
        # print the source on the left
        for row in range(len(DistTable)):
            if row != 0:
                print(source[row-1], end="")
            else:
                print(" ", end="")
            # print the minimum edit distance cost table
            for value in DistTable[row]:
                print(f'{value : >3}', end="") # Padding of 3, aligned right
            print()

    return DistTable # Returns the minimum edit distance value at the bottom right of the table

file = costsCSV
TestTable = MED("devious", "mischievous", file)


      m  i  s  c  h  i  e  v  o  u  s  
   0  1  2  3  4  5  6  7  8  9 10 11
d  1  2  3  4  5  6  7  8  9 10 11 12
e  2  3  4  5  6  7  8  7  8  9 10 11
v  3  4  5  6  7  8  9  8  7  8  9 10
i  4  5  4  5  6  7  8  9  8  9 10 11
o  5  6  5  6  7  8  9 10  9  8  9 10
u  6  7  6  7  8  9 10 11 10  9  8  9
s  7  8  7  6  7  8  9 10 11 10  9  8


In [961]:
# Backtracing

import random

backtracePathPositions = []  # (x,y)
backtracePathOperations = [] # ("d", "i", "s", "k")

def shortestPath(MEDTable): 
    return MEDTable[-1][-1] # Return the bottom right cell cost, which is the shortest distance

def backtrace(MEDTable):
    # Make sure backtrace arrays are emptied
    backtracePathPositions = []
    backtracePathOperations = []

    if False:
        for row in MEDTable:
            for cell in row:
                print(f'{cell : >2}', end=" ")
            print()
        print(savedSource, savedTarget)

    x = len(MEDTable[0])-1
    y = len(MEDTable)-1

    while (x > -1 or y > -1): #while current position is not at the origin (start from bottom right of table)
        # using or because table edges need to be processed
        currentTargetLetter = x-1
        currentSourceLetter = y-1

        randomQueue = []

        # Cell positions [x,y]
        currentCell = MEDTable[y][x]

        diagonalCell = MEDTable[y-1][x-1]
        horizontalCell = MEDTable[y][x-1]
        verticalCell = MEDTable[y-1][x]

        # Debug every step
        if False:
            print("XY: ", x, y)
            
            print("Curr letter pos: ", currentSourceLetter, currentTargetLetter)
            print("Curr letter s t: ", savedSource[currentSourceLetter], savedTarget[currentTargetLetter])

            print(diagonalCell, verticalCell)
            print(horizontalCell, currentCell)

            print("same?", savedSource[currentSourceLetter] == savedTarget[currentTargetLetter],\
                ", diag-2?", currentCell-2 == diagonalCell,\
                ", horz-1?", currentCell-1 == horizontalCell,\
                ", vert-1?", currentCell-1 == verticalCell)

        sameLetter = savedSource[currentSourceLetter] == savedTarget[currentTargetLetter] #only evaluate once
        if sameLetter and x > 0 and y > 0: #if diag == same letter, append diag to backtracePath and move on #this probably causes outofbounds issue by hopping over border
            randomQueue.append("k")
            #print("k added to random queue")
        # If we're traversing the border
        elif x == 0 and y > 1:
            randomQueue.append("d")
        elif x > 1 and y == 0:
            randomQueue.append("i")
        #elif x == 1 and y == 1:
            #randomQueue.append("s")
        elif not sameLetter: #and not on edge 
            if currentCell-2 == diagonalCell or currentCell == diagonalCell:# and currentSourceLetter > 0 and currentTargetLetter > 0: #if here -2 == diag, add to random queue #will need to update to work with confusing matrix
                randomQueue.append("s")
                #print("s added to random queue")
            if currentCell-1 == horizontalCell: #and currentSourceLetter > 0: #if here -1 == left, add to random queue
                randomQueue.append("i")
                #print("i added to random queue")
            if currentCell-1 == verticalCell: #and currentTargetLetter > 0: #if here -1 == up, add to random queue
                randomQueue.append("d")
                #print("d added to random queue")

        #print("End at 0,0:", currentSourceLetter, currentTargetLetter)
        #if currentSourceLetter == 0 and currentTargetLetter == 0:
        if x == 0 and y == 0: #?
            trueBacktracePathOperations = backtracePathOperations[::-1] # Reverse array
            #print(backtracePathPositions)
            #print(backtracePathOperations)
            return trueBacktracePathOperations

        # Choose random path
        backtracePathPositions.append((x,y))
        randomChoice = random.choice(randomQueue) #choose random path
        #print("RandomQueue: ", randomQueue, "Choice: ", randomChoice)
        backtracePathOperations.append(randomChoice)
        

        # Traverse backtrace
        if randomChoice == "k" or randomChoice == "s":
            #print("keep/substitute", savedTarget[currentTargetLetter])
            x-=1
            y-=1
        elif randomChoice == "d":
            #print("insert", savedTarget[currentTargetLetter])
            y-=1
        elif randomChoice == "i":
            #print("delete", savedSource[currentSourceLetter])
            x-=1

        
        
    print()

    # Print debug info if the loop fails
    print("XY: ", x, y)
            
    print("Curr letter pos: ", currentSourceLetter, currentTargetLetter)
    print("Curr letter s t: ", savedSource[currentSourceLetter], savedTarget[currentTargetLetter])

    print(diagonalCell, verticalCell)
    print(horizontalCell, currentCell)

    print("same?", savedSource[currentSourceLetter] == savedTarget[currentTargetLetter])
    print("diag-2?", currentCell-2 == diagonalCell)
    print("horz-1?", currentCell-1 == horizontalCell)
    print("vert-1?", currentCell-1 == verticalCell)

    print(savedSource, savedTarget)
    print(backtracePathPositions)
    print(backtracePathOperations)
    return -1

backtrace(TestTable) #Testing
#shortestPath()

#bugs to fix: 
#s not appearing at end of devious final out
#e->v marked as keep in devious final out


['i', 'i', 'i', 'i', 'd', 's', 'i', 's', 's', 'k', 'k', 'k']

In [962]:
# This block of code outputs the results of the targets and sources

for pair in pairTargSources: # mischevious, execution
    #print("TARGET: ", pair[0])
    for sourceList in pair[1:]: # [mischief, devious], [intention]
        #print("SOURCELIST: ", sourceList, end="\n")
        for singleSource in sourceList: # mischief, devious
            #print("SOURCE WORD: ", singleSource)
            for csv in costCSVs: # costsFileName, costs2FileName
                #print("Levenshtein" if costMethod == costsFile else "Confusion Matrix")
                MEDTable = MED(singleSource, pair[0], csv)
                backtraceOperations = backtrace(MEDTable) # Each cost method has to do a backtrace
                if backtraceOperations == -1:
                    print("Backtrace error")
                    break
                print()

                # print source word
                sourceIndex = 0
                for oper in backtraceOperations:
                    if oper == "i":
                        print("* ", end="")
                    elif sourceIndex <= len(singleSource):
                        print(singleSource[sourceIndex], "", end="")
                        sourceIndex += 1

                print()

                # print vertical bars
                for oper in range(len(backtraceOperations)):
                    print("| ", end="")

                print()

                # print target word
                targetIndex = 0
                for oper in backtraceOperations:
                    if oper == "d":
                        print("* ", end="")
                    elif targetIndex <= len(pair[0]):
                        print(pair[0][targetIndex], "", end="")
                        targetIndex += 1

                print()

                # print backtrace operations and shortest path
                for oper in range(len(backtraceOperations)):
                    print(backtraceOperations[oper], end=" ")
                print("(",shortestPath(MEDTable),")")
                print()

            # Print 50 hyphens to separate source words
            print("-" * 50)

      d  u  p  e  d  
   0  1  2  3  4  5
d  1  0  1  2  3  4
u  2  1  0  1  2  3
k  3  2  1  2  3  4
e  4  3  2  3  2  3
d  5  4  3  4  3  2

d u k e d 
| | | | | 
d u p e d 
k k s k k ( 2 )

      d  u  p  e  d  
   0  1  2  3  4  5
d  1  0  1  2  3  4
u  2  1  0  1  2  3
k  3  2  1  0  1  2
e  4  3  2  1  0  1
d  5  4  3  2  1  0

d u k e d 
| | | | | 
d u p e d 
k k s k k ( 0 )

--------------------------------------------------
      m  i  s  c  h  i  e  v  o  u  s  
   0  1  2  3  4  5  6  7  8  9 10 11
m  1  0  1  2  3  4  5  6  7  8  9 10
i  2  1  0  1  2  3  4  5  6  7  8  9
s  3  2  1  0  1  2  3  4  5  6  7  8
c  4  3  2  1  0  1  2  3  4  5  6  7
h  5  4  3  2  1  0  1  2  3  4  5  6
i  6  5  4  3  2  1  0  1  2  3  4  5
e  7  6  5  4  3  2  1  0  1  2  3  4
f  8  7  6  5  4  3  2  1  2  3  4  5

m i s c * h i * e * f * 
| | | | | | | | | | | | 
m i s c h i e v o u * s 
k k k k i s s i s i d i ( 5 )

      m  i  s  c  h  i  e  v  o  u  s  
   0  1  2  3  4  5  6  7  8  9 10